In [0]:
#Descargar datos de Dropbox
!wget  https://www.dropbox.com/s/5abxkrmi43g5sdl/ucmerced.zip?dl=1 -O ucmerced.zip
!unzip -x ucmerced.zip

--2019-07-25 16:52:17--  https://www.dropbox.com/s/5abxkrmi43g5sdl/ucmerced.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/5abxkrmi43g5sdl/ucmerced.zip [following]
--2019-07-25 16:52:17--  https://www.dropbox.com/s/dl/5abxkrmi43g5sdl/ucmerced.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6f5278cf5391b09a667cdcee49.dl.dropboxusercontent.com/cd/0/get/AlWZKX_TbPEBIfg4ex62Brlldaw-HRfW7rNIVV58G53VuCSaU4SshizZKpaQj0PWk3DI5917AGGhKL9o0IZwgAhMkBZhZP4WKR8iguJcsvBdJQ/file?dl=1# [following]
--2019-07-25 16:52:18--  https://uc6f5278cf5391b09a667cdcee49.dl.dropboxusercontent.com/cd/0/get/AlWZKX_TbPEBIfg4ex62Brlldaw-HRfW7rNIVV58G53VuCSaU4SshizZKpaQj0PWk3DI5917AGGhKL9o0IZwgAhMkBZhZP4WKR8iguJcsvBdJQ/file?dl=1
Resolving uc

In [0]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from keras.preprocessing import image
from keras.models import Model
import numpy as np
import os
import pandas as pd
import subprocess

red='resnet50'
#red='vgg16'

if red=='resnet50':
  from keras.applications.resnet50 import ResNet50
  from keras.applications.resnet50 import preprocess_input
  base_model = ResNet50(weights='imagenet')
  model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)
  
if red=='vgg16':
  from keras.applications.vgg16 import VGG16
  from keras.applications.vgg16 import preprocess_input
  base_model = VGG16(weights='imagenet')
  model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc2').output)

def training(X_train,y_train):
  caracteristicas_train = deep_extractor(X_train)
  X_train = np.array(caracteristicas_train)
  # Entrenar SVM
  clf = LinearSVC(C=1000000,max_iter=5000)
  clf.fit(X_train, y_train)
  return clf

def deep_extractor(imagenes):
  caracteristicas = []
  for i in tqdm(imagenes):
    img = image.load_img( i , target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    fully_connected = model.predict(x)
    caracteristicas.append(fully_connected[0])
  return caracteristicas

def test(X_test,y_test):
  caracteristicas_test = deep_extractor(X_test)
  X_test = np.array(caracteristicas_test)
  return X_test,y_test

def test_score(clasificador,X_test,y_test):
  return clasificador.score(X_test,y_test)

def split_dataset(L,metodo):
  clases = list()
  imagenes = list()
  for root, dirs, files in os.walk(os.path.abspath(L+'/Images')) :
      for file in files:
          if file.endswith(".tif"):
            clases.append(str(os.path.basename(os.path.normpath(root))))
            imagenes.append(os.path.join(root, file))

  # Dividir train, val y test
  if metodo=='supervisado':
    X_train, X_test, y_train, y_test = train_test_split(imagenes, clases, test_size=0.2)
    X_train, X_val, y_train , y_val = train_test_split(X_train, y_train, test_size=0.25)
    
    print("train: ",len(X_train))
    print("  val: ",len(X_val))
    print(" test: ",len(X_test))
    print("total: ",len(X_train)+len(X_val)+len(X_test))

    return X_train, X_val, X_test, y_train, y_val, y_test
    
  if metodo=='semi-supervisado':
    L_img, U_img, L_class, U_class = train_test_split(imagenes, clases, test_size=0.5)
    X_train, X_test, y_train, y_test = train_test_split(L_img, L_class, test_size=0.4)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.6666666)

    U_img = U_img[:int(len(U_img)*porcentaje)]
    U_class_ = U_class[:int(len(U_class)*porcentaje)]

    print('\n')
    print("train: ",len(X_train))
    print("  val: ",len(X_val))
    print(" test: ",len(X_test))
    print("U_img: ",len(U_img))
    print("total etiquetado: ",len(X_train)+len(X_val)+len(X_test))
    print("total no-etiquetado: ",len(U_img))
    print("total global: ",len(X_train)+len(X_val)+len(X_test)+len(U_img))
  
    return X_train, X_val, X_test, y_train, y_val, y_test, U_img, U_class
  
def return_batch_set(U_img,batch_size):
  return np.split(np.array(U_img), int(1/batch_size))

def update_training(X_train,y_train,EL):
  [X_train.append(i[0]) for i in EL]
  [y_train.append(i[1]) for i in EL]
  caracteristicas_train = deep_extractor(X_train)
  X_train = np.array(caracteristicas_train)
  # Entrenar SVM
  clf = LinearSVC(C=1000000,max_iter=5000)
  clf.fit(X_train, y_train)
  return clf

metodo='semi-supervisado'
L = 'ucmerced'
porcentaje=1

# Usando SVM Scikit-learn
X_train, X_val, X_test, y_train, y_val, y_test, U_img, U_class = split_dataset(L,metodo)
clasificador = training(X_train,y_train)
X_test, y_test = test(X_test,y_test)
print('\n')
print('Test_score_'+red+': ',test_score(clasificador,X_test,y_test))

Using TensorFlow backend.
W0725 16:52:58.578058 140661037696896 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0725 16:52:58.619198 140661037696896 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0725 16:52:58.632859 140661037696896 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0725 16:52:58.666733 140661037696896 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session inste

102858752/102853048 [==============================] - 1s 0us/step


  0%|          | 0/210 [00:00<?, ?it/s]



train:  210
  val:  420
 test:  420
U_img:  1050
total etiquetado:  1050
total no-etiquetado:  1050
total global:  2100


100%|██████████| 420/420 [00:04<00:00, 87.77it/s]



Test_score_resnet50:  0.8333333333333334
